In [7]:
#This code was run on Windows

import tkinter as tk
import tkinter.messagebox
import collections
import random
import math
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt

SMALL_FONT = ("Arial",12)
SMALL_BOLD = ("Arial",12,"bold")
MEDIUM_BOLD = ("Arial",16,"bold")

PARAMETER_NAME1 = ["Price (P)", "Coupon Payment(C)", "Yield to Maturity(I/Y)", "Years to Maturity(T)"] 
PARAMETER_NAME2 = ["Current stock price (S0)", "Annual interest rate (r in %)", "Volatility (sigma, 0<sigma<1)",
                        "Maturity of option (T in years)", "Strike price (K)", "Number of paths to simulate(N)"]

ErrMsg1 = "Invalid inputs! Please enter 3 non-negative parameters and select the payment method."
ErrMsg2 = "Invalid inputs! Please enter correct parameters and select type of the option."

AUTHOR_INFO = "Version 1.4\nBy Shawn\nyz3380@columbia.edu"


def Errormsg(msg):
    tk.messagebox.showwarning("Error",msg)

    
class PricingHelper(tk.Tk): # base class

    def __init__(self, master=None):
        tk.Tk.__init__(self)
        self.title("PricingHelper v1.4")
        self.geometry("420x450")
        self._frame = None
        self.switch_frame(IndexPage) # set the default page to index

    def switch_frame(self, next_frame): # destory current page and create a new one
        new_frame = next_frame(self)
        if self._frame is not None:
            self._frame.destroy()
        self._frame = new_frame
        self._frame.pack()

class IndexPage(tk.Frame): 
     
    def __init__(self, master):
        tk.Frame.__init__(self, master)
 
        Name = tk.Label(self, text="Pricing Helper", font=("Arial",32,'bold'), fg="blue")
        Name.grid(row=0, column=0, sticky="NWES", pady=20)
        
        Menu = tk.Label(self, text="Select a function:", font=("Arial",16))
        Menu.grid(row=1, column=0, sticky="NWES", pady=10)
        
        GoBond = tk.Button(self, text="Bond Pricing", font=SMALL_BOLD, width=20, height=2, 
                           command=lambda: master.switch_frame(BondPage))
        GoBond.grid(row=2, column=0, pady=5)
        
        GoOption = tk.Button(self, text="Option Pricing", font=SMALL_BOLD, width=20, height=2, 
                             command=lambda: master.switch_frame(OptionPage))
        GoOption.grid(row=3, column=0, pady=5)
        
        Quit = tk.Button(self, text="QUIT", fg="red", font=SMALL_BOLD, width=20, height=2, command=lambda:app.destroy())
        Quit.grid(row=4, column=0, pady=5)
        
        Author = tk.Label(self, text=AUTHOR_INFO, font=("Arial", 8), fg="grey")
        Author.grid(row=5, column=0, sticky="NWES", pady=20)
        
        
class BondPage(tk.Frame):
    
    def __init__(self, master):
        tk.Frame.__init__(self, master)
        
        Container = collections.defaultdict(list) # a dictionary to store all widgets
        
        Name = tk.Label(self, text="Bond Value Calculator", font=("Arial",24,"bold"), fg="blue")
        Name.grid(row=0, column=0, columnspan=6, pady=15)
        Instruction = tk.Label(self, text="Enter any 3 parameters (Face value is 1000)", font=SMALL_FONT, anchor="n")
        Instruction.grid(row=1, column=0, columnspan=6, pady=5)
        
        for i,parameter in enumerate(PARAMETER_NAME1):
            label = tk.Label(self, text=parameter, font=SMALL_FONT, width=16, anchor="w")
            label.grid(row=i+2, column=0, columnspan=2, pady=6, padx=10)
        
        # save all the entries as a list in container
        for i in range(4):
            var = tk.DoubleVar() 
            entry = tk.Entry(self, font=("Arial",12), textvar=var, width=15, bd=5, bg='white')
            Container["Entry"].append(entry)
            entry.grid(row=i+2, column=2, columnspan=2)
        
        Percent1 = tk.Label(self, text="$", font= SMALL_FONT, anchor="w", width=2)
        Percent1.grid(row=3,column=4,sticky="W")
        Percent2 = tk.Label(self, text="%", font= SMALL_FONT, anchor="w", width=2)
        Percent2.grid(row=4,column=4,sticky="W")
        
        # check if a zero coupon bond, saved in container
        Chkval = tk.IntVar() 
        Container["Zerocheck"]= tk.Checkbutton(self, text="Zero", variable=Chkval, font=SMALL_FONT, anchor="w",
                                    command=lambda:self.zero_coupon(Container))
        Container["Zerocheck"].grid(row=3, column=5, sticky="W")
        
        Payment_method = tk.Label(self, text="Payment method", font=SMALL_FONT, width=15, anchor="w")
        Payment_method.grid(row=6, column=0, columnspan=2, pady=5)
        
        # save type of payment method
        Container["Paymethod"] = tk.Listbox(self, selectmode="SINGLE", bd=3, height=4)
        Container["Paymethod"].insert(1, "Annually")
        Container["Paymethod"].insert(2, "Semi-annually")
        Container["Paymethod"].insert(3, "Quarterly")
        Container["Paymethod"].insert(4, "Monthly")
        Container["Paymethod"].grid(row=6, column=2, rowspan=2, columnspan=2, pady=5)
        
        # calculate based on the inputs
        Calculate = tk.Button(self, text="Calculate", font=MEDIUM_BOLD, fg="white", bg="light green", width=9,
                            command=lambda:self.cal(Container, Chkval)) 
        Calculate.grid(row=8, column=2, columnspan=2, pady=5)
        
        # reset all the inputs
        Clear = tk.Button(self, text="Reset", font=MEDIUM_BOLD, fg="white", bg="dark grey",
                            command=lambda: self.clear(Container))
        Clear.grid(row=8, column=4, columnspan=2, pady=5)
        
        Back = tk.Button(self, text="Back", font=SMALL_FONT, width=10, command=lambda: master.switch_frame(IndexPage))
        Back.grid(row=9, column=0, columnspan=2, pady=7)
        
        Quit = tk.Button(self, text="QUIT", fg="red", font=SMALL_FONT, width=8, command=lambda:app.destroy())
        Quit.grid(row=9, column=4, columnspan=2, pady=7)
    
    
    def cal(self, container, chkval):
        
        values = [] # check string values and empty inputs
        for i in range(4): 
            if not container["Entry"][i].get(): values.append(0) # transform null value(the entry left blank) into zero
            else: # check if all the inputs are numerical
                try:
                    values.append(float(container["Entry"][i].get()))
                except Exception:
                    Errormsg(ErrMsg1)
                    return

        if chkval.get()==1: values[1]=0
    
        nvar = sum([x>0 for x in [values[0],values[2],values[3]]])
        positive = sum([x<0 for x in values])
        
        # at least two of the parameters must be positive except for coupon, 
        # payment method must be selected, and none of the parameter should be negative
        if (nvar<2 or positive>0 or len(container["Paymethod"].curselection())==0):  
                Errormsg(ErrMsg1)
        else:        
            Npayments = [1,2,4,12]
            n = Npayments[container["Paymethod"].curselection()[0]] # get the annual payment time 
            price, coupon, rate, period = values

            if price == 0: # price is 0, calculate current price(present value)
                try:
                    price = np.pv(rate=rate/(100*n), nper=period*n, pmt=coupon, fv=1000)
                    price = round(abs(price),2)
                    container["Entry"][0].delete(0,tk.END)
                    container["Entry"][0].insert(0,price)
                    self.infomsg("Price of bond is $", price)
                    self.intprt(price, coupon, rate, period, n)
                except Exception as e:
                    Errormsg(e)
            
            elif rate == 0: # rate is 0, lets calculate rate of return
                try:
                    rate = np.rate(nper=period*n, pmt=coupon, pv=-price, fv=1000)
                    rate = round(rate*n*100,2)
                    container["Entry"][2].delete(0,tk.END)
                    container["Entry"][2].insert(0,rate)
                    self.infomsg("Annual yield of bond is ", rate)
                    self.intprt(price, coupon, rate, period, n)
                except Exception as e:
                    Errormsg(e)
            
            elif period == 0: # period is 0, lets calculate number of periods to maturity
                try:
                    period = np.nper(rate=rate/(100*n), pmt=-coupon, pv=price, fv=-1000)
                    period = round(period/n,2)
                    
                    if period <= 0 or math.isnan(period): # check for meaningless output or values not defined
                        Errormsg("This bond theoretically does not exist.\nPlease check your inputs.")
                    else:
                        container["Entry"][3].delete(0,tk.END)
                        container["Entry"][3].insert(0,period)
                        self.infomsg("Years to maturity of bond is ", period)
                        self.intprt(price, coupon, rate, period, n)
                except Exception as e:
                    Errormsg(e)
            
            else: # all other inputs are positive, calculate the coupon rate
                try:
                    coupon = np.pmt(rate=rate/(100*n), nper=period*n, pv=-price, fv=1000)
                    coupon = abs(round(coupon,2))
                    container["Entry"][1].delete(0,tk.END)
                    container["Entry"][1].insert(0,coupon)
                    self.infomsg("Coupon payment of bond is ", coupon)
                    self.intprt(price, coupon, rate, period, n)
                except Exception as e:
                    Errormsg(e)


    def zero_coupon(self, container): # clear the coupon rate if it is a zero coupon bond
        container["Entry"][1].delete(0,tk.END)
        container["Entry"][1].insert(0,0)
    
    def clear(self, container):
        for i in range(4):
            container["Entry"][i].delete(0,tk.END)
            container["Entry"][i].insert(0,0.0)
        container["Paymethod"].selection_clear(0, 2)
        container["Zerocheck"].deselect()  
        
    def infomsg(self, msg1, number):
        msg = msg1 + str(number)
        tk.messagebox.showinfo("Calculation Result", msg)
    
    def intprt(self, price, coupon, rate, period, n): # interpret the calculation result directly at console
        pay_freq = ["annually","semi-annually","quarterly","monthly"]
        freq = pay_freq[[1,2,4,12].index(n)]
        coupon = coupon*n/1000*100
        print("A $1000 face value bond with annual coupon rate {}%, paid {}, matures in {} years,".format(coupon,freq,period))
        print("is priced at ${} at the initial date given the annual interest rate is {}%.\n".format(price,rate))
        

class OptionPage(tk.Frame):
    
    def __init__(self, master):      
        tk.Frame.__init__(self, master)
        
        Container = collections.defaultdict(list) # a dictionary to store all widgets
        
        Name = tk.Label(self, text="Option Price Simulator", font=("Arial",24,"bold"), fg="blue")
        Name.grid(row=0, column=0, columnspan=3, pady=15)
        Instruction = tk.Label(self, text="Set all parameters to get the price of an European option", font=("Arial",10))
        Instruction.grid(row=1, column=0, columnspan=3)
        
        for i,parameter in enumerate(PARAMETER_NAME2):
            label = tk.Label(self, text=parameter, font=SMALL_FONT, width=24, anchor="w")
            label.grid(row=i+2, column=0, columnspan=2, pady=5, padx=10)
        
        for i in range(6):
            var = tk.DoubleVar() 
            entry = tk.Entry(self, font=("Arial",12), textvar=var, width=12, bd=5, bg='white')
            Container["Entry"].append(entry)
            entry.grid(row=i+2, column=2)
        
        Payment_method = tk.Label(self, text="Option type", font=SMALL_FONT, width=24, anchor="w")
        Payment_method.grid(row=8, column=0, columnspan=2, pady=5)
        
        # save type of the option
        Container["Type"] = tk.Listbox(self, selectmode="SINGLE", width=16, bd=3, height=2)
        Container["Type"].insert(1, "Call Option")
        Container["Type"].insert(2, "Put Option")
        Container["Type"].grid(row=8, column=2)
        
        Pricetag = tk.Label(self, text="Price:", font=SMALL_BOLD, width=12)
        Pricetag.grid(row=9, column=0, pady=15)
        
        Container["Price"] = tk.Label(self, text="$ 0.0", font=SMALL_BOLD, width=12,anchor="w")
        Container["Price"].grid(row=9, column=1)
        
        Get_price = tk.Button(self, text="Get Price!", font=MEDIUM_BOLD, fg="white", bg="light green",
                            command=lambda:self.getprice(Container)) 
        Get_price.grid(row=9, column=2)
        
        Back = tk.Button(self, text="Back", font=SMALL_FONT, width=10, command=lambda: master.switch_frame(IndexPage))
        Back.grid(row=10, column=0, pady=5)
        
        Quit = tk.Button(self, text="QUIT", fg="red", font=SMALL_FONT, width=10, command=lambda:app.destroy())
        Quit.grid(row=10, column=2, pady=5)
    
    
    def getprice(self, container):
                   
        values = [] # check string values and empty inputs
        for i in range(6): 
            if not container["Entry"][i].get(): values.append(0) # transform null value(the entry left blank) into zero
            else: # check if all the inputs are numerical
                try:
                    values.append(float(container["Entry"][i].get()))
                except Exception:
                    Errormsg(ErrMsg2)
                    return
    
        positive = any (x<=0 for x in values[2:5])
        # all parameters must be positive except for interest rate r, 
        if (positive or values[0]<=0 or values[1]<0 or values[2]>=1 or len(container["Type"].curselection())==0):  
            Errormsg(ErrMsg2)
        else: 
            optype = container["Type"].curselection()[0]        
            price = self.simulate(optype, values)
            container["Price"].config(text = "$ "+ str(price))
    
    
    def simulate(self, optype, values): # simulate the option price
        
        s0, r, sigma, T, K, N = values
        r = r/100       
        # start to generate paths of option prices using Monto Carlo simulation
        path = []
        for i in range(int(N)):
            s = s0
            # we assume the stock price follows a geometric Brownian motion
            s = s*math.exp((r - 0.5*sigma**2)*T + sigma*math.sqrt(T)*random.gauss(0,1))
            payoff = max(s-K,0) if optype==0 else max(K-s,0)
            payoff = math.exp(-r*T)*payoff # convert to current value
            path.append(payoff)
        
        price = np.mean(path)
        
        t_price = self.Black_Scholes(s0, K, r, sigma, T, optype)
        self.intpr(price, t_price, path) # interpret the result
        return round(price,3)
    
    def Black_Scholes(self, s, K, r, sigma, T, optype):
        # calculate the theoretical estimate of the price of the option using Black-Scholes formula
        d1 = (1/(sigma*math.sqrt(T)))*(math.log(s/K)+T*(r+0.5*sigma**2))
        d2 = d1 - sigma*math.sqrt(T)
        price = st.norm.cdf(d1)*s - st.norm.cdf(d2)*K*math.exp(-r*T)
        
        # use put-call parity to update price of put option
        if optype == 1: 
            price = K*math.exp(-r*T)-s+price
        
        return round(price,3)
    
    def intpr(self, price, t_price, path):
        # plot the cumulative average of path and check if it converges to theoretical value
        average = np.cumsum(path)
        for i in range(len(path)):
            average[i] = average[i]/(i+1)   
 
        plt.plot(average)
        plt.ylim(0,max(max(average),t_price)+1)
        plt.axhline(y=t_price, linestyle='dashed')
        title = "Simulation Result of Option Price = $"+str(round(price,3))+"\n"
        title = title + "Theoretical Estimated Price is $"+str(t_price)      
        plt.title(title)
        plt.xlabel("Path")
        plt.ylabel("Price")
        plt.show()
        tk.messagebox.showinfo("Simulation result","Price of option is $"+str(round(price,3)))

        

app = PricingHelper()
app.mainloop()
